In [16]:
require 'nn';

In [17]:
net = nn.Sequential()

-- 3 input image channel
-- 96 output channels
-- 11x11 convolution kernel
-- stride = 4 in both dimensions
net:add(nn.SpatialConvolution(3, 96, 11, 11, 4, 4))

-- non-linearity 
net:add(nn.ReLU())

-- Max-pool with 3x3 window with stride 2
net:add(nn.SpatialMaxPooling(3,3,2,2))     

-- LRN with size = 5
-- alpha = 0.0001 and beta = 0.75 are default values
net:add(nn.SpatialCrossMapLRN(5))


--[[
96 input channels
256 output channels
5x5 convolution kernel
stride = 1 in both dimensions
padding = 2 in both dimensions
groups = 2
groups thing taken from https://github.com/soumith/cudnn.torch/blob/master/SpatialConvolution.lua
Not sure if it will work
]]
net:add(nn.SpatialConvolution(96, 256, 5, 5, 1, 1, 2, 2, 2))

-- non-linearity 
net:add(nn.ReLU())

-- Max-pool with 3x3 window with stride 2
net:add(nn.SpatialMaxPooling(3,3,2,2)) 

-- LRN with size = 5
-- alpha = 0.0001 and beta = 0.75 are default values
net:add(nn.SpatialCrossMapLRN(5))

--[[
256 input channels
384 output channels
3x3 convolution kernel
stride = 1 in both dimensions
padding = 1 in both dimensions
]]
net:add(nn.SpatialConvolution(256, 384, 3, 3, 1, 1, 1, 1))

-- non-linearity 
net:add(nn.ReLU())

--[[
384 input channels
384 output channels
3x3 convolution kernel
stride = 1 in both dimensions
padding = 1 in both dimensions
groups = 2
groups thing taken from https://github.com/soumith/cudnn.torch/blob/master/SpatialConvolution.lua
Not sure if it will work
]]
net:add(nn.SpatialConvolution(384, 384, 3, 3, 1, 1, 1, 1, 2))

-- non-linearity 
net:add(nn.ReLU())

--[[
384 input channels
256 output channels
3x3 convolution kernel
stride = 1 in both dimensions
padding = 1 in both dimensions
groups = 2
groups thing taken from https://github.com/soumith/cudnn.torch/blob/master/SpatialConvolution.lua
Not sure if it will work
]]
net:add(nn.SpatialConvolution(384, 256, 3, 3, 1, 1, 1, 1, 2))

-- non-linearity 
net:add(nn.ReLU())

-- Max-pool with 3x3 window with stride 2
net:add(nn.SpatialMaxPooling(3,3,2,2))

-- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
net:add(nn.View(256*6*6))    

-- Fully Connected
net:add(nn.Linear(9216, 4096))   

-- non-linearity 
net:add(nn.ReLU())

-- dropout
net:add(nn.Dropout(0.5))

-- Fully Connected
net:add(nn.Linear(4096, 42))

-- converts the output to a log-probability. Useful for classification problems
net:add(nn.LogSoftMax())   

print('AttributeNet\n' .. net:__tostring());

AttributeNet
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> output]
  (1): nn.SpatialConvolution(3 -> 96, 11x11, 4,4)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(3x3, 2,2)
  (4): nn.SpatialCrossMapLRN
  (5): nn.SpatialConvolution(96 -> 256, 5x5, 1,1, 2,2)
  (6): nn.ReLU
  (7): nn.SpatialMaxPooling(3x3, 2,2)
  (8): nn.SpatialCrossMapLRN
  (9): nn.SpatialConvolution(256 -> 384, 3x3, 1,1, 1,1)
  (10): nn.ReLU
  (11): nn.SpatialConvolution(384 -> 384, 3x3, 1,1, 1,1)
  (12): nn.ReLU
  (13): nn.SpatialConvolution(384 -> 256, 3x3, 1,1, 1,1)
  (14): nn.ReLU
  (15): nn.SpatialMaxPooling(3x3, 2,2)
  (16): nn.View(9216)
  (17): nn.Linear(9216 -> 4096)
  (18): nn.ReLU
  (19): nn.Dropout(0.500000)
  (20): nn.Linear(4096 -> 42)
  (21): nn.LogSoftMax
}	


In [18]:
net:evaluate()
input = torch.rand(3,227,227);
output = net:forward(input);

In [19]:
conv = nn.Sequential()
for i=1,17 do
    conv:add(net.modules[i]:clone())
end
print(conv:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> output]
  (1): nn.SpatialConvolution(3 -> 96, 11x11, 4,4)
  (2): nn.ReLU
  (3): nn.SpatialMaxPooling(3x3, 2,2)
  (4): nn.SpatialCrossMapLRN
  (5): nn.SpatialConvolution(96 -> 256, 5x5, 1,1, 2,2)
  (6): nn.ReLU
  (7): nn.SpatialMaxPooling(3x3, 2,2)
  (8): nn.SpatialCrossMapLRN
  (9): nn.SpatialConvolution(256 -> 384, 3x3, 1,1, 1,1)
  (10): nn.ReLU
  (11): nn.SpatialConvolution(384 -> 384, 3x3, 1,1, 1,1)
  (12): nn.ReLU
  (13): nn.SpatialConvolution(384 -> 256, 3x3, 1,1, 1,1)
  (14): nn.ReLU
  (15): nn.SpatialMaxPooling(3x3, 2,2)
  (16): nn.View(9216)
  (17): nn.Linear(9216 -> 4096)
}	


In [63]:
mlp1=nn.Sequential();
mlp1:add(nn.Linear(100,10));
mlp1:add(nn.Linear(10,1));

-- clone and share the parameters
mlp2 = mlp1:clone('weight','bias', 'gradWeight','gradBias')

mlpMain:add(mlp2)

-- we change the bias of the first
mlp1:get(2).bias[1] = 99;

-- and see that the second one's bias has also changed..
print(mlp2:get(2).bias[1])

99	


In [65]:
-- ?nn.DotProduct